In [117]:

using ProgressMeter


In [118]:
h = 0.2; x0 = -1.0; xn = 1.0; activation = relu; dim = 1; N1 = 10_000
# We create some data (normalized)
x = hcat(range(x0,stop=xn,step=h)...)
f = x->exp(sin(x*pi))
delay = length(x)*1.2
y = map(f,x)

model = Chain(DenseNTK(dim=>N1,activation),DenseNTK(N1=>dim))|>f64
detail = 1
iterations = 10000

params_0 = deepcopy(Flux.params(model))
println("Approximately $delay sec delay do to kernel calculations")


Loss(model,x,y) = Flux.mse(y,model(x))
LR_opt2,K,eig = LR_updt(h,x0,xn,detail,iterations)



Approximately 13.2 sec delay do to kernel calculations

Progress:  15%|███████                                  |  ETA: 0:00:01

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


(4.976113593635157, [1.962605605428523 1.7379616820107138 … 0.3969238563292167 0.32479015850406284; 1.7379616820107138 1.6093558587171668 … 0.46051460513293363 0.39736829694161213; … ; 0.3969238563292167 0.46051460513293363 … 1.6290532365444605 1.7602360493713836; 0.32479015850406284 0.39736829694161213 … 1.7602360493713836 1.9929575443757326], [0.015708408303290525, 0.017501314569513305, 0.019041288207687888, 0.02259800020596566, 0.028214695084646343, 0.03868730577591074, 0.06068680416428707, 0.11525399478779252, 0.429462213251862, 3.885235663527463, 10.60098742894941])

In [3]:

L = []
y = map(f,x)
data = [(x,y)]
@showprogress for epoch = 1:iterations

    Flux.train!(Loss,model,data,Descent(LR_opt2))
    push!(L,Loss(model,x,y))

end 

Progress: 100%|█████████████████████████████████████████| Time: 0:03:14


In [4]:
L

100000-element Vector{Any}:
 0.5018901333648181
 0.43824358181176826
 0.4019024957743511
 0.379514528628699
 0.364807999814988
 0.35440826250008156
 0.3464587678537305
 0.33993299591042075
 0.3342470466359757
 0.3290886342912216
 ⋮
 5.968192169830961e-28
 5.968192169830961e-28
 5.968192169830961e-28
 5.968192169830961e-28
 5.968192169830961e-28
 5.968360250989744e-28
 5.984031017693601e-28
 5.984031017693601e-28
 6.033155537700546e-28

In [6]:
using JLD2


In [7]:
save_object("Params2.jld2",Flux.params(model))